# Dependencies installation

Automatically installs all dependencies along with their imports.

In [2]:
%pip install tensorflow
%pip install pandas numpy scikit-learn scikit-metrics
%pip install tensorflow_datasets
%pip install -q mlflow

import os
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras import layers, models # type: ignore

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mlflow

mlflow.login()

# Dataset pre-processing

Imports the datasets, while preprocessing and normalizing it.

In [3]:
url = "https://raw.githubusercontent.com/timothypesi/Data-Sets-For-Machine-Learning-/main/california_housing_train.csv"
dataset = pd.read_csv(url)

In [4]:
train_df, test_df = train_test_split(dataset, test_size=0.8, shuffle=True)

In [5]:
train_y = train_df.pop("median_house_value")
test_y = test_df.pop("median_house_value")

In [ ]:
normalizer = tf.keras.layers.Normalization()

normalizer.adapt(train_df)
normalizer.adapt(test_df)

In [19]:
mlflow.set_experiment("/artifacts/mlflow-housing")

mlflow.tensorflow.autolog()

model = tf.keras.models.Sequential()

model.add(normalizer)
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(200, activation="relu"))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.05), loss=tf.keras.losses.MeanAbsoluteError()) #Watch out for the loss function

model.fit(train_df, train_y, epochs=300)

#test with other layers and with more epochs, and try adjusting the learning rate

Epoch 1/300
107/107 [==============================] - 1s 2ms/step - loss: 73491.7656
Epoch 2/300
107/107 [==============================] - 0s 2ms/step - loss: 52554.9336
Epoch 3/300
107/107 [==============================] - 0s 2ms/step - loss: 49579.4336
Epoch 4/300
107/107 [==============================] - 0s 2ms/step - loss: 50998.6641
Epoch 5/300
107/107 [==============================] - 0s 2ms/step - loss: 48083.0352
Epoch 6/300
107/107 [==============================] - 0s 2ms/step - loss: 48049.6133
Epoch 7/300
107/107 [==============================] - 0s 2ms/step - loss: 48101.8164
Epoch 8/300
107/107 [==============================] - 0s 2ms/step - loss: 47701.6289
Epoch 9/300
107/107 [==============================] - 0s 2ms/step - loss: 46518.3867
Epoch 10/300
107/107 [==============================] - 0s 2ms/step - loss: 46437.4102
Epoch 11/300
107/107 [==============================] - 0s 2ms/step - loss: 46513.5820
Epoch 12/300
107/107 [==============================

In [18]:
model.evaluate(test_df, test_y)

425/425 [==============================] - 1s 1ms/step - loss: 37388.5312


37388.53125

In [20]:
model.save("artifacts/house_price_model")

INFO:tensorflow:Assets written to: artifacts/house_price_model/assets


INFO:tensorflow:Assets written to: artifacts/house_price_model/assets


In [ ]:
prediction = model.predict(test_df)

test_df["predicted_median_house_value"] = prediction
test_df["median_house_value"] = test_y